# Power Point Presentation maker

## Configs

In [ ]:
from __future__ import annotations

import os

from dotenv import load_dotenv
from openai import OpenAI
from pydantic import BaseModel

load_dotenv()
api_key = os.environ.get("OPENAI_API_KEY")
if not api_key:
    msg = "Missing API_KEY"
    raise OSError(msg)

## Pydantic class as OpenAI's expected response format

[OpenAI docs - structured format](https://platform.openai.com/docs/guides/structured-outputs)

In [ ]:

class SingleSlide(BaseModel):
    title: str
    content: str
    visual: str


class ResponseFormat(BaseModel):
    slide: list[SingleSlide]

In [ ]:
from prompts import sys_prompt, usr_prompt

client = OpenAI(api_key=api_key)
completion = client.beta.chat.completions.parse(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": sys_prompt},
        {"role": "user", "content": usr_prompt},
    ],
    response_format=ResponseFormat,
)
event = completion.choices[0].message.parsed

In [ ]:
for i in event.slide:
    print(f"{i.title}\n{i.content}\n{i.visual}\n")

List of Day-to-Day AI Tools for Developers
An overview of essential AI tools that can enhance productivity and efficiency in development tasks.
image of diverse AI tools

Introduction to AI Tools for Developers
The rapid evolution of AI technology has provided developers with a plethora of tools to streamline workflows, troubleshoot coding issues, and enhance overall project outcomes. This presentation will discuss some of the most useful AI tools available today and how they can be integrated into your daily coding practices.
introductory graphic of AI technology

Search Tools: Perplexity
Perplexity is an advanced search tool that leverages AI to provide relevant search results quickly. Ideal for developers looking for quick answers and references during coding.
screenshot of Perplexity interface

Research Tools: NotebookLM
NotebookLM helps developers organize research and collaborate on notes seamlessly. It integrates coding environments with documentation, making it easier to access

## Function to create images with Stable Diffusion

Using a local Stable Diffusion API to create the images, currently WebUI Forge.

In [ ]:
import base64
from pathlib import Path

import requests

# Define the URL and the payload to send.
url = "http://127.0.0.1:7860"


def imagecreator(prompt: str, name: str, width: int, height: int) -> None:
    """Access local Stable Diffusion to create images.

    Args:
        prompt (str): prompt for image creation
        name (str): Name of the resulting image
        width (int): Image Width
        height (int): Image Height

    """
    payload = {
        "prompt": prompt,
        "steps": 20,
        "sampler_name": "DPM++ 2M",
        "scheduler": "Beta",
        "cfg_scale": 1,
        "width": width,
        "height": height,
    }

    # Send said payload to said URL through the API.
    response = requests.post(url=f"{url}/sdapi/v1/txt2img", json=payload, timeout= 300)
    r = response.json()

    # Decode and save the image.
    with Path(f"images/{name}.png").open("wb") as f:
        f.write(base64.b64decode(r["images"][0]))

## Create a PPTX with the responses generated with ChatGPT and the Stable Diffusion images

Using [Python-pptx](https://python-pptx.readthedocs.io/en/latest/index.html) for PowerPoint creation

In [ ]:
from pptx import Presentation
from pptx.util import Inches

prs = Presentation()
bullet_slide_layout = prs.slide_layouts[1]

for i in event.slide:
    slide = prs.slides.add_slide(bullet_slide_layout)
    shapes = slide.shapes

    # Add background image
    left = top = Inches(0)
    slide_width = prs.slide_width
    slide_height = prs.slide_height

    imagecreator(i.visual + "NO TEXT", i.title, 955, 537)
    pic = slide.shapes.add_picture(
        str(Path(f"images/{i.title}.png")), left, top, slide_width, slide_height,
    )

    # Add Text
    title_shape = shapes.title
    body_shape = shapes.placeholders[1]

    title_shape.text = i.title

    tf = body_shape.text_frame
    tf.text = i.content


prs.save("test.pptx")